In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler




In [22]:
df = pd.read_csv("train.csv")

In [23]:
df = df.drop(columns=['ID'])
df = df.drop(columns=['Payment Plan'])
df = df.drop(columns=['Accounts Delinquent'])
df = df.drop(columns=['Batch Enrolled'])
df =df.drop(columns=['Application Type'])

In [24]:
df['Loan Title'] = df['Loan Title'].str.lower()
len(df['Loan Title'].unique())
df['Loan Title'].unique()

array(['debt consolidation', 'credit card refinancing',
       'home improvement', 'credit consolidation', 'green loan', 'other',
       'moving and relocation', 'credit cards', 'medical expenses',
       'refinance', 'credit card consolidation', 'lending club',
       'debt consolidation loan', 'major purchase', 'vacation',
       'business', 'credit card payoff', 'credit card',
       'credit card refi', 'personal loan', 'cc refi', 'consolidate',
       'medical', 'loan 1', 'consolidation', 'card consolidation',
       'car financing', 'debt', 'home buying', 'freedom', 'consolidated',
       'get out of debt', 'consolidation loan', 'dept consolidation',
       'personal', 'cards', 'bathroom', 'refi', 'credit card loan',
       'credit card debt', 'house', 'debt consolidation 2013',
       'debt loan', 'cc refinance', 'home', 'cc consolidation',
       'credit card refinance', 'credit loan', 'payoff',
       'bill consolidation', 'credit card paydown', 'credit card pay off',
       'g

In [25]:
map_dict = {'Loan Title':{'debt consolidation':'debt consolidation', \
                'debt consolidation loan':'debt consolidation', \
                'debt consolidation 2013':'debt consolidation', \
                'dept consolidation':'debt consolidation' ,
                'credit card refinancing':'credit card refinancing', \
                'refinance':'credit card refinancing', \
                'cc refinance':'credit card refinancing',\
                'credit card refinance':'credit card refinancing',\
                'cc-refinance':'credit card refinancing', \
                'refinance loan':'credit card refinancing', \
                'credit card refinance loan':'credit card refinancing',\
                'credit card refi':'credit card refinancing', \
                'cc refi':'credit card refinancing', \
                'refi':'credit card refinancing'  ,
                'home improvement':'home improvement',\
                'home improvement loan':'home improvement',\
                'bathroom':'home improvement',\
                'pool':'home improvement' ,
                'home buying':'home buy',\
                'house':'home buy',\
                'home':'home buy',\
                'home loan':'home buy' ,
                'credit consolidation':'credit consolidation',\
                'consolidation loan':'credit consolidation',\
                'cc consolidation':'credit consolidation',\
                'conso':'credit consolidation',\
                'credit card consolidation':'credit consolidation',\
                'card consolidation':'credit consolidation',\
                'loan consolidation':'credit consolidation',\
                'consolidate':'credit consolidation',\
                'consolidated':'credit consolidation' ,
                'medical':'medical purpose',\
                'medical expenses':'medical purpose',\
                'medical loan':'medical purpose'  ,
                'lending club' : 'lending purpose' , \
                'lending loan' : 'lending purpose'  ,
                'business' : 'business purpose' , \
                'green loan' : 'business purpose' , \
                'moving and relocation' : 'business purpose'  , \
                'credit cards' : 'credit cards payoff' , \
                'credit card' : 'credit cards payoff' , \
                'credit card loan' : 'credit cards payoff' , \
                'credit' : 'credit cards payoff' , \
                'credit card debt' : 'credit cards payoff' , \
                'cc loan' : 'credit cards payoff' , \
                'cc' : 'credit cards payoff' , \
                'cards' : 'credit cards payoff' , \
                'credit card payoff' : 'credit cards payoff' , \
                'credit pay off' : 'credit cards payoff' , \
                'credit payoff' : 'credit cards payoff' , \
                'credit loan' : 'credit cards payoff' , \
                'payoff' : 'credit cards payoff' , \
                'pay off' : 'credit cards payoff' , \
                'credit card paydown' : 'credit cards payoff' , \
                'credit card pay off' : 'credit cards payoff' ,
                'personal loan' : 'perosonal purpose' , \
                'personal' : 'perosonal purpose' , \
                'wedding loan' : 'perosonal purpose' , \
                'getting ahead' : 'perosonal purpose' , \
                'vacation' : 'perosonal purpose' , \
                'major purchase' : 'perosonal purpose'  , \
                'car financing' : 'vehicle purpose' , \
                'car loan' : 'vehicle purpose' , \
                'debt' : 'debt payoff' , \
                'get out of debt' : 'debt payoff' , \
                'debt loan' : 'debt payoff' , \
                'get debt free' : 'debt payoff' , \
                'debt payoff' : 'debt payoff' , \
                'debt free' : 'debt payoff' , \
                'freedom' : 'debt payoff' , \
                'relief' : 'debt payoff' , \
                'debt reduction' : 'debt payoff' , \
                'bill consolidation' : 'bill payoff' , \
                'bill payoff' : 'bill payoff' , \
                'pay off bills' : 'bill payoff' , \
                'bills' : 'bill payoff'  , \
                'other' : 'other purpose' , \
                'loan 1' : 'other purpose' , \
                'loan1' : 'other purpose' , \
                'loan' : 'other purpose' , \
                'myloan' : 'other purpose' , \
                'my loan' : 'other purpose' }
            }

In [26]:
df.replace(map_dict, inplace=True)
len(df['Loan Title'].unique())


15

In [27]:

colunas = ['Home Ownership', 'Interest Rate', 'Inquires - six months', 'Total Accounts',
               'Total Received Interest', 'Total Received Late Fee', 'Recoveries', 
               'Collection Recovery Fee', 'Collection 12 months Medical', 
               'Total Collection Amount', 'Total Revolving Credit Limit']

#Loop em relação as features
for coluna in colunas:
  Q1 = df[coluna].quantile(q = 0.25) #Definindo o primeiro quartil
  Q3 = df[coluna].quantile(q = 0.75) #Definindo o segundo quartil
  IQR = Q3 - Q1 #Definindo o interquartil
  superior = Q3 + IQR  #Definindo o Limite Superior
  inferior = Q1 - IQR #Definindo o Limite Inferior

  #Eliminando os outliers acima do limite superior
  df.drop( df.loc[ df[coluna] > superior ].index, axis = 0, inplace = True )

  #Eliminando os outliers abaixo do limite inferior
  df.drop( df.loc[ df[coluna] < inferior ].index, axis = 0, inplace = True )

In [28]:
df.drop(['Collection 12 months Medical', 'Inquires - six months'], axis = 1, inplace = True)
print(df.head())

    Loan Amount  Funded Amount  Funded Amount Investor  Term  Interest Rate  \
1          3609          11940            12191.996920    59      12.237563   
7         20744          10609             7645.014802    58      13.993688   
8          9299          11238            13429.456610    59      11.178457   
9         19232           8962             7004.097481    58       5.520413   
11        16581           8767            10637.049030    59       9.535988   

   Grade Sub Grade Employment Duration  Home Ownership Verification Status  \
1      C        D3                RENT     39833.92100     Source Verified   
7      A        A5                 OWN     61723.52014        Not Verified   
8      G        C2            MORTGAGE     63205.09072            Verified   
9      C        B5                RENT     42015.46586     Source Verified   
11     A        D4            MORTGAGE     39605.50605     Source Verified   

    ... Initial List Status  Total Received Interest  To

In [29]:
import pandas as pd
df=pd.read_csv("train_with_dummies_0_1.csv")
X = df.drop(['Loan Status'], axis = 1)
y = df['Loan Status']

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.neighbors import KNeighborsClassifier

#Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle = True, 
                                                    stratify = y_train,
                                                    random_state = 42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#Normalização
for col in X:
    feature_range = (X_train[col].min(), X_train[col].max())
    
    if feature_range[0] < 0 or feature_range[1] > 1:
        scaler = MinMaxScaler()
        X_train[col] = scaler.fit_transform(X_train[[col]])
        X_test[col] = scaler.transform(X_test[[col]])

knn_model = KNeighborsClassifier(n_neighbors=10)  # Número de vizinhos

knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Precisão: {accuracy}")
print("\nMatriz de Confusão:")
print(conf_matrix)
print("\nRelatório de Classificação:")
print(class_report)


Precisão: 0.5038045654785743

Matriz de Confusão:
[[795 482]
 [757 463]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.51      0.62      0.56      1277
           1       0.49      0.38      0.43      1220

    accuracy                           0.50      2497
   macro avg       0.50      0.50      0.49      2497
weighted avg       0.50      0.50      0.50      2497



In [12]:

svm_model = SVC()  

svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
class_report_svm = classification_report(y_test, y_pred_svm)

print("Resultados do Modelo SVM:")
print(f"Precisão: {accuracy_svm}")
print("\nMatriz de Confusão:")
print(conf_matrix_svm)
print("\nRelatório de Classificação:")
print(class_report_svm)


Resultados do Modelo SVM:
Precisão: 0.5062074489387265

Matriz de Confusão:
[[641 636]
 [597 623]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.52      0.50      0.51      1277
           1       0.49      0.51      0.50      1220

    accuracy                           0.51      2497
   macro avg       0.51      0.51      0.51      2497
weighted avg       0.51      0.51      0.51      2497



In [13]:
from sklearn.naive_bayes import GaussianNB

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

nb_model = GaussianNB() 
nb_model.fit(X_train, y_train)

y_pred_nb = nb_model.predict(X_test)

accuracy_nb = accuracy_score(y_test, y_pred_nb)
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
class_report_nb = classification_report(y_test, y_pred_nb)

print(f"Precisão (Naive Bayes): {accuracy_nb}")
print("\nMatriz de Confusão (Naive Bayes):")
print(conf_matrix_nb)
print("\nRelatório de Classificação (Naive Bayes):")
print(class_report_nb)


Precisão (Naive Bayes): 0.5158189827793352

Matriz de Confusão (Naive Bayes):
[[1143  134]
 [1075  145]]

Relatório de Classificação (Naive Bayes):
              precision    recall  f1-score   support

           0       0.52      0.90      0.65      1277
           1       0.52      0.12      0.19      1220

    accuracy                           0.52      2497
   macro avg       0.52      0.51      0.42      2497
weighted avg       0.52      0.52      0.43      2497



In [14]:
from sklearn.linear_model import LogisticRegression

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo de Regressão Logística
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

accuracy_lr = accuracy_score(y_test, y_pred_lr)
conf_matrix_lr = confusion_matrix(y_test, y_pred_lr)
class_report_lr = classification_report(y_test, y_pred_lr)

print(f"Precisão (Logistic Regression): {accuracy_lr}")
print("\nMatriz de Confusão (Logistic Regression):")
print(conf_matrix_lr)
print("\nRelatório de Classificação (Logistic Regression):")
print(class_report_lr)


Precisão (Logistic Regression): 0.5102122547056468

Matriz de Confusão (Logistic Regression):
[[531 746]
 [477 743]]

Relatório de Classificação (Logistic Regression):
              precision    recall  f1-score   support

           0       0.53      0.42      0.46      1277
           1       0.50      0.61      0.55      1220

    accuracy                           0.51      2497
   macro avg       0.51      0.51      0.51      2497
weighted avg       0.51      0.51      0.51      2497



In [15]:
from sklearn.ensemble import BaggingClassifier

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo Bagging Classifier
bagging_model = BaggingClassifier()
bagging_model.fit(X_train, y_train)

y_pred_bagging = bagging_model.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
conf_matrix_bagging = confusion_matrix(y_test, y_pred_bagging)
class_report_bagging = classification_report(y_test, y_pred_bagging)

print(f"Precisão (Bagging Classifier): {accuracy_bagging}")
print("\nMatriz de Confusão (Bagging Classifier):")
print(conf_matrix_bagging)
print("\nRelatório de Classificação (Bagging Classifier):")
print(class_report_bagging)


Precisão (Bagging Classifier): 0.49939927913496196

Matriz de Confusão (Bagging Classifier):
[[782 495]
 [755 465]]

Relatório de Classificação (Bagging Classifier):
              precision    recall  f1-score   support

           0       0.51      0.61      0.56      1277
           1       0.48      0.38      0.43      1220

    accuracy                           0.50      2497
   macro avg       0.50      0.50      0.49      2497
weighted avg       0.50      0.50      0.49      2497



In [16]:
from sklearn.ensemble import RandomForestClassifier

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print(f"Precisão (Random Forest): {accuracy_rf}")
print("\nMatriz de Confusão (Random Forest):")
print(conf_matrix_rf)
print("\nRelatório de Classificação (Random Forest):")
print(class_report_rf)


Precisão (Random Forest): 0.5190228273928714

Matriz de Confusão (Random Forest):
[[674 603]
 [598 622]]

Relatório de Classificação (Random Forest):
              precision    recall  f1-score   support

           0       0.53      0.53      0.53      1277
           1       0.51      0.51      0.51      1220

    accuracy                           0.52      2497
   macro avg       0.52      0.52      0.52      2497
weighted avg       0.52      0.52      0.52      2497



In [17]:
from sklearn.ensemble import AdaBoostClassifier

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo AdaBoost
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(X_train, y_train)

y_pred_adaboost = adaboost_model.predict(X_test)

accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
conf_matrix_adaboost = confusion_matrix(y_test, y_pred_adaboost)
class_report_adaboost = classification_report(y_test, y_pred_adaboost)

print(f"Precisão (AdaBoost): {accuracy_adaboost}")
print("\nMatriz de Confusão (AdaBoost):")
print(conf_matrix_adaboost)
print("\nRelatório de Classificação (AdaBoost):")
print(class_report_adaboost)


Precisão (AdaBoost): 0.527032438926712

Matriz de Confusão (AdaBoost):
[[653 624]
 [557 663]]

Relatório de Classificação (AdaBoost):
              precision    recall  f1-score   support

           0       0.54      0.51      0.53      1277
           1       0.52      0.54      0.53      1220

    accuracy                           0.53      2497
   macro avg       0.53      0.53      0.53      2497
weighted avg       0.53      0.53      0.53      2497



In [18]:
from sklearn.ensemble import GradientBoostingClassifier

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo Gradient Boosting
gradientboost_model = GradientBoostingClassifier()
gradientboost_model.fit(X_train, y_train)

y_pred_gradientboost = gradientboost_model.predict(X_test)

accuracy_gradientboost = accuracy_score(y_test, y_pred_gradientboost)
conf_matrix_gradientboost = confusion_matrix(y_test, y_pred_gradientboost)
class_report_gradientboost = classification_report(y_test, y_pred_gradientboost)

print(f"Precisão (Gradient Boosting): {accuracy_gradientboost}")
print("\nMatriz de Confusão (Gradient Boosting):")
print(conf_matrix_gradientboost)
print("\nRelatório de Classificação (Gradient Boosting):")
print(class_report_gradientboost)


Precisão (Gradient Boosting): 0.527032438926712

Matriz de Confusão (Gradient Boosting):
[[614 663]
 [518 702]]

Relatório de Classificação (Gradient Boosting):
              precision    recall  f1-score   support

           0       0.54      0.48      0.51      1277
           1       0.51      0.58      0.54      1220

    accuracy                           0.53      2497
   macro avg       0.53      0.53      0.53      2497
weighted avg       0.53      0.53      0.53      2497



In [19]:
from xgboost import XGBClassifier

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo XGBoost
xgboost_model = XGBClassifier()
xgboost_model.fit(X_train, y_train)

y_pred_xgboost = xgboost_model.predict(X_test)

accuracy_xgboost = accuracy_score(y_test, y_pred_xgboost)
conf_matrix_xgboost = confusion_matrix(y_test, y_pred_xgboost)
class_report_xgboost = classification_report(y_test, y_pred_xgboost)

print(f"Precisão (XGBoost): {accuracy_xgboost}")
print("\nMatriz de Confusão (XGBoost):")
print(conf_matrix_xgboost)
print("\nRelatório de Classificação (XGBoost):")
print(class_report_xgboost)


Precisão (XGBoost): 0.5166199439327193

Matriz de Confusão (XGBoost):
[[654 623]
 [584 636]]

Relatório de Classificação (XGBoost):
              precision    recall  f1-score   support

           0       0.53      0.51      0.52      1277
           1       0.51      0.52      0.51      1220

    accuracy                           0.52      2497
   macro avg       0.52      0.52      0.52      2497
weighted avg       0.52      0.52      0.52      2497



In [20]:
import lightgbm as lgb

# Instanciando o objeto RandomUnderSampler()
rus = RandomUnderSampler()

X_train, y_train = rus.fit_resample(X, y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, 
                                                    y_train, 
                                                    shuffle=True, 
                                                    stratify=y_train,
                                                    random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Modelo LightGBM
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)

y_pred_lgb = lgb_model.predict(X_test)

accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
conf_matrix_lgb = confusion_matrix(y_test, y_pred_lgb)
class_report_lgb = classification_report(y_test, y_pred_lgb)

print(f"Precisão (LightGBM): {accuracy_lgb}")
print("\nMatriz de Confusão (LightGBM):")
print(conf_matrix_lgb)
print("\nRelatório de Classificação (LightGBM):")
print(class_report_lgb)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5021, number of negative: 4964
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000706 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4213
[LightGBM] [Info] Number of data points in the train set: 9985, number of used features: 78
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502854 -> initscore=0.011417
[LightGBM] [Info] Start training from score 0.011417
Precisão (LightGBM): 0.5206247496996396

Matriz de Confusão (LightGBM):
[[624 653]
 [544 676]]

Relatório de Classificação (LightGBM):
              precision    recall  f1-score   support

           0       0.53      0.49      0.51      1277
           1       0.51      0.55      0.53      1220

    accuracy                           0.52      2497
   macro avg       0.52      0.52      0.52      2497
weighted avg       0.52 